In [38]:
import sys
import os
import math
import random

import pandas as pd
import numpy as np
#import plotnine as p9

Specify path and file name of file containing list of file names with metadata:

In [41]:
filenames_path = r"C:\Users\m229246\OneDrive - AZCollaboration\Documents\!Other Data\!2015 - Pharm Dev\!Work Items\Brilinta Particle Size\Data\!TEXT FILES - RAW DATA/"
#file_path = r"C:\Users\m229246\OneDrive - AZCollaboration\Documents\!Other Data\!2015 - Pharm Dev\!Work Items\Brilinta Particle Size\Data\!TEXT FILES - RAW DATA/"
                                         
filenames_name = "Raw data & meta data.xlsx"
df_names = pd.read_excel(filenames_path + filenames_name,sheet_name='Data for Python DEMO')
#print(df_names)


In [42]:
#print(df_names)

# Create one stacked file combining all individual files

In [43]:
# See list of the file names:
for i, j in df_names.iterrows():
    file_name = j['Filename']
    #print(j)
    print(file_name)

C:\Users\m229246\OneDrive - AZCollaboration\Documents\!Other Data\!2015 - Pharm Dev\!Work Items\Brilinta Particle Size\Data\!TEXT FILES - RAW DATA\Bril04.txt
C:\Users\m229246\OneDrive - AZCollaboration\Documents\!Other Data\!2015 - Pharm Dev\!Work Items\Brilinta Particle Size\Data\!TEXT FILES - RAW DATA\Bril06.txt
C:\Users\m229246\OneDrive - AZCollaboration\Documents\!Other Data\!2015 - Pharm Dev\!Work Items\Brilinta Particle Size\Data\!TEXT FILES - RAW DATA\Bril07.txt


In [44]:
# Loop through the rows of file "df_names":
for i, j in df_names.iterrows():
    file_name = j['Filename']
    
    # Create a list "Skip" specifying random rows to omit leaving a sample size of 5000:
    n = sum(1 for line in open(file_name)) - 1     #number of records in file (excludes header)
    s = 5000 #desired sample size
    skip = sorted(random.sample(range(1,n+1),n-s)) #the 0-indexed header will not be included in the skip list
    
    # Read each file skipping the random rows and creating a new variable "Batch":
    df = pd.read_csv(file_name,sep='\t', encoding='latin1', skiprows=skip)
    df['Batch'] = j['Batch']

    # Stack the current file to the last created ones:
    if (i==0):
        appended_data = pd.concat([df], ignore_index=True)
    else:
        appended_data = pd.concat([appended_data,df], ignore_index=True)

In [45]:
df = appended_data

In [46]:
print(df)

           Id  Magnification  ^CE Diameter (µm)  Length (µm)  Width (µm)  \
0       32184            5.0             150.79      1154.51      169.54   
1       55809            5.0              63.93       150.16       36.30   
2       49322            5.0              54.86       138.11       91.87   
3       12455            5.0              54.06       178.40       50.11   
4       71749            5.0              53.34       126.59       56.01   
...       ...            ...                ...          ...         ...   
14995   17301            5.0               2.17         3.44        1.79   
14996  130434            5.0               2.17         2.92        1.92   
14997   34101            5.0               2.17         3.44        1.79   
14998   77092            5.0               2.17         3.23        2.00   
14999   31586            5.0               2.17         3.13        1.96   

       Max. Distance (µm)  Perimeter (µm)  Major Axis (°)  Area (µm²)  \
0             

In [31]:
#df.to_excel(filenames_path + 'df_test_all.xlsx')

# Summarise the data by binning

In [32]:
# Create a new variable which is log cylinder volume:
df["cyl_vol"]=math.pi*df["Length (µm)"]*(df["Width (µm)"]/2)**2
df["log_cyl_vol"]=np.log10(df["cyl_vol"])

# Create new variable "bin centres":
df["bins"] = pd.cut(df['log_cyl_vol'], bins=np.linspace(0, 10, 41))
df["bin_centres"] = df["bins"].apply(lambda x: x.mid)

# Create a df with medians by bin:
df_summ = df.groupby(['Batch','bin_centres'])[["Circularity","HS Circularity","Convexity","Solidity","Aspect Ratio","Elongation","Intensity Mean","Intensity SD"]].median()
df_summ = df_summ.reset_index()
df_summ = df_summ.dropna(subset=['Circularity'])

# Create a df with particle counts by bin:
df_summ2 = df.groupby(['Batch','bin_centres'])[["Id"]].count()
df_summ2 = df_summ2.reset_index()
df_summ2 = df_summ2.dropna(subset=['Id'])

# Create a df with sum of cylinder volume by bin:
df_summ3 = df.groupby(['Batch','bin_centres'])[["cyl_vol"]].sum()
df_summ3 = df_summ3.reset_index()
df_summ3 = df_summ3.dropna(subset=['cyl_vol'])

# Merge together: df_summ, df_summ2 and df_summ3:
df_comb = df_summ2.merge(df_summ, how='outer', on=['Batch','bin_centres'])
df_comb2 = df_comb.merge(df_summ3, how='outer', on=['Batch','bin_centres'] )
df_comb2 = df_comb2.rename(columns = {'Id':'Count'})
df_comb2 = df_comb2.rename(columns = {'cyl_vol':'Sum of cylinder volume'})

In [33]:
print(df_comb2)

   Batch bin_centres  Count  Circularity  HS Circularity  Convexity  Solidity  \
0    B04       0.625    1.0       0.9010          0.8120     1.0000    1.0000   
1    B04       0.875   44.0       0.8435          0.7115     1.0000    1.0000   
2    B04       1.125  167.0       0.8520          0.7260     1.0000    1.0000   
3    B04       1.375  220.0       0.8495          0.7215     1.0000    1.0000   
4    B04       1.625  249.0       0.8390          0.7040     1.0000    1.0000   
..   ...         ...    ...          ...             ...        ...       ...   
65   B07       4.875   32.0       0.3495          0.1220     0.7565    0.4145   
66   B07       5.125   11.0       0.3090          0.0960     0.6960    0.3200   
67   B07       5.375    7.0       0.3260          0.1070     0.7160    0.3940   
68   B07       5.625    1.0       0.3580          0.1280     0.9560    0.4950   
69   B07       6.375    1.0       0.2390          0.0570     0.8320    0.2700   

    Aspect Ratio  Elongatio

In [34]:
#df_comb2['cum_sum_cyl_vol'] = df_comb2.groupby('Batch')['Sum of cylinder volume'].transform('cumsum')
#df_comb2['cum_perc_cyl_vol_1'] = 100*df_comb2['cum_sum_cyl_vol'] / df_comb2.groupby('Batch')['Sum of cylinder volume'].transform('sum')

# Create 1st Excel file for Spotfire - binned data

Calculate % cylinder volume and cumulative % per batch
Similarly for count of particles

In [35]:
df_comb2['perc_cyl_vol'] = 100*df_comb2['Sum of cylinder volume'] / df_comb2.groupby('Batch')['Sum of cylinder volume'].transform('sum')
df_comb2['cum_perc_cyl_vol'] = df_comb2.groupby('Batch')['perc_cyl_vol'].transform('cumsum')

In [36]:
df_comb2['perc_N'] = 100*df_comb2['Count'] / df_comb2.groupby('Batch')['Count'].transform('sum')
df_comb2['cum_perc_N'] = df_comb2.groupby('Batch')['perc_N'].transform('cumsum')

In [37]:
print(df_comb2)

   Batch bin_centres  Count  Circularity  HS Circularity  Convexity  Solidity  \
0    B04       0.625    1.0       0.9010          0.8120     1.0000    1.0000   
1    B04       0.875   44.0       0.8435          0.7115     1.0000    1.0000   
2    B04       1.125  167.0       0.8520          0.7260     1.0000    1.0000   
3    B04       1.375  220.0       0.8495          0.7215     1.0000    1.0000   
4    B04       1.625  249.0       0.8390          0.7040     1.0000    1.0000   
..   ...         ...    ...          ...             ...        ...       ...   
65   B07       4.875   32.0       0.3495          0.1220     0.7565    0.4145   
66   B07       5.125   11.0       0.3090          0.0960     0.6960    0.3200   
67   B07       5.375    7.0       0.3260          0.1070     0.7160    0.3940   
68   B07       5.625    1.0       0.3580          0.1280     0.9560    0.4950   
69   B07       6.375    1.0       0.2390          0.0570     0.8320    0.2700   

    Aspect Ratio  Elongatio

Merge with df_names to join with meta data:

In [20]:
df_comb2 = df_comb2.merge(df_names, how='outer', on=['Batch'])
df_comb2 = df_comb2.drop(['Filename'], axis=1)

Output to Excel file

In [23]:
df_comb2.to_excel(filenames_path + 'df_comb2b DEMO.xlsx')

# Create 2nd Excel file for Modde - pivot of binned data

Count data:

In [72]:
df_count = pd.pivot_table(df_comb2, values='Count', index=['Batch'],columns='bin_centres')
df_count = df_count.add_prefix('Count_')

Shape data:

In [73]:
df_shape1 = pd.pivot_table(df_comb2, values='Circularity', index=['Batch'],columns='bin_centres')
df_shape1 = df_shape1.add_prefix('Circularity_')
df_shape2 = pd.pivot_table(df_comb2, values='HS Circularity', index=['Batch'],columns='bin_centres')
df_shape2 = df_shape2.add_prefix('HS_Circularity_')
df_shape3 = pd.pivot_table(df_comb2, values='Convexity', index=['Batch'],columns='bin_centres')
df_shape3 = df_shape3.add_prefix('Convexity_')
df_shape4 = pd.pivot_table(df_comb2, values='Solidity', index=['Batch'],columns='bin_centres')
df_shape4 = df_shape4.add_prefix('Solidity_')
df_shape5 = pd.pivot_table(df_comb2, values='Aspect Ratio', index=['Batch'],columns='bin_centres')
df_shape5 = df_shape5.add_prefix('Aspect_Ratio_')
df_shape6 = pd.pivot_table(df_comb2, values='Elongation', index=['Batch'],columns='bin_centres')
df_shape6 = df_shape6.add_prefix('Elongation_')
df_shape7 = pd.pivot_table(df_comb2, values='Intensity Mean', index=['Batch'],columns='bin_centres')
df_shape7 = df_shape7.add_prefix('Intensity_Mean_')
df_shape8 = pd.pivot_table(df_comb2, values='Intensity SD', index=['Batch'],columns='bin_centres')
df_shape8 = df_shape8.add_prefix('Intensity_SD_')


In [74]:
from functools import partial, reduce

In [75]:
dfshapes = [df_shape1, df_shape2, df_shape3, df_shape4, df_shape5, df_shape6, df_shape7, df_shape8]
merge = partial(pd.merge, on=['Batch'], how='outer')
dfshapes = reduce(merge, dfshapes)

In [76]:
dfshapes = dfshapes.merge(df_names, how='outer', on=['Batch'])
dfshapes = dfshapes.drop(['Filename'], axis=1)

Export pivoted data to Excel:

In [77]:
dfshapes.to_excel(filenames_path + 'df_shape DEMO.xlsx')